# Tensorflow Demo
playing with the [Deep ConvNet for MNIST digit classification](https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html#build-a-multilayer-convolutional-network) tutorial

## Import MNIST data

In [ ]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets('MNIST-data', one_hot=True)

## simple softmax regression demo

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

# set input/output placeholders
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
# define the model parameters as tf.Variable's
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# initialize them in our InteractiveSession
sess.run(tf.initialize_all_variables())

y = tf.nn.softmax(tf.matmul(x,W) + b)

# Loss function = the discrete cross entropy between the predicted distributions and the labels 1-hot vectors
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# use steepest descent to minmize loss function, 
# train_step is an op that will compute gradient, compute update step, apply update
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# So to find min, just repeated run the train_step
for i in range(1000):
    if i % 100 == 0:
        print i
    batch = mnist.train.next_batch(50)
    # note, using feed_dict can replace any tensor in graph, not just tf.placeholders
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
# evaluate
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accu = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

print(accu.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

sess.close()

In [ ]:
import tensorflow as tf
import numpy as np

A = np.random.random([3,3])
print(A)
b = np.array([[1],[2],[3]]).astype(float)
sess = tf.Session()
print(sess.run(tf.constant(A)+tf.constant(b)))

## Deep ConvNet demo

In [ ]:
import tensorflow as tf

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


## Data preprocessing script

In [6]:
import numpy as np
import datetime
import itertools
import cPickle
from collections import Counter
import re
"""
Code adapted from https://github.com/dennybritz/cnn-text-classification-tf
"""

def clean_str(string):
    """
    Tokenization/string cleaning for input.
    Originally taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = tokenize(string)
    return string.strip().lower()

def tokenize(string):
    """
    Tokenize English strings
    """
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ( ", string) 
    string = re.sub(r"\)", " ) ", string) 
    string = re.sub(r"\?", " ? ", string) 
    string = re.sub(r"\s{2,}", " ", string) 
    return string
    
def normalize_sentence_length(sentences, padding_word='<PAD/>', max_length_words = None):
    """
    Normalizes lengths to max_length. max_length = None, normalizes to max sentence length.
    Pads shorter sentences using the padding_word. 
    Cuts longer sentences at max_length number of words.
    
    sentences - list of sentence where each sentence is a list of words. eg. [['foo','bar'], ['fo2','bar2']]
    """
    max_length_words = max_length_words if max_length_words is not None else max(len(x) for x in sentences)
    norm_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = max_length_words - len(sentence)
        padded_sentence = sentence + [padding_word] * num_padding
        chopped_sentence = padded_sentence[:max_length_words]
        norm_sentences.append(chopped_sentence)
    return norm_sentences

def raw_text_to_sentence(text):
    return clean_str(text).split(" ")

def build_vocab(sentences):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    print('building vocab...')
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

def build_input_data(sentences, labels, vocabulary):
    """
    Maps sentences to vectors based on vocabulary.
    Maps binary labels to 1 hot vectors, 0 -> [1,0], 1 -> [0,1]
    returns np.arrays
    """
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    y = np.array([[0,1] if label else [1,0] for label in labels])

def read_pos_and_neg_data(header=True):
    with open('./data/raw_input_neg.tsv', 'rt') as f:
        f.readline() if header else None
        lines = f.readlines()
        negatives = [(0, line.split('\t')[3]) for line in lines]
        print('num of negative examples: {}'.format(len(negatives)))
    with open('./data/raw_input_pos.tsv', 'rt') as f:
        f.readline() if header else None
        lines = f.readlines()
        positives = [(1, line) for line in lines]
        print('num of positive examples: {}'.format(len(positives)))
    return positives + negatives

def write_dev_train_sets(label_texts, path_template, p=0.1):
    """
    Outputs a dev data set using apprx p percent of the list of label and text tuples label_texts
    Rest goes to the training set
    """
    print('saving dev and training data sets...')
    # sample p% for dev use 
    dev_mask = np.random.rand(len(label_texts)) <= p
    train_mask = np.array([not d for d in dev_mask])
    full = np.array(label_texts)
    dev = full[dev_mask]
    train = full[train_mask]
    
    with open(path_template.format('dev'), 'wt') as df:
        for pair in dev:
            df.write(str.format('{}\t{}\n', pair[0], pair[1]))
    with open(path_template.format('train'), 'wt') as tf:
        for pair in train:
            tf.write(str.format('{}\t{}\n', pair[0], pair[1]))

def preprocess_raw_inputs_and_save():
    labels, texts = zip(*read_pos_and_neg_data())
    sentences = map(raw_text_to_sentence, texts)
    normalized_sentences = normalize_sentence_length(sentences)
    vocab, inv_vocab = build_vocab(normalized_sentences)
    
    ts = datetime.datetime.now().strftime('%Y-%m-%d.%H%M%S')
    path_template = './data/{{}}_set.{}'.format(ts)
    with open('./data/vocab.pickle.{}'.format(ts), 'wt') as vocab_file:
        pickle.dump({'vocabulary': vocab, 'inv_vocabulary': inv_vocab}, vocab_file)
    write_dev_train_sets(zip(labels, [" ".join(sent) for sent in normalized_sentences]), path_template, p=0.1)
    
def load_input_data(example_path, vocab_path):
    with open(example_path, 'rt') as f:
        lines = f.readlines()
        labels = [line.split('\t')[0] for line in lines]
        texts = [line.split('\t')[1] for line in lines]
        sentences = map(raw_text_to_sentence, texts)
    with open(vocab_path, 'rb') as v:
        vocab_map = pickle.load(v)
    return build_input_data(sentences, labels, vocab_map['vocabulary'])

preprocess_raw_inputs_and_save()

[['adsfadsf', 'adsfa', 'adsfa', 'dfd'],
 ['asdf', 'asd', 'f', 'asdfsfd', 'sdfad']]

## CNN class to define computation graph

In [5]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

[2, 3]